In [15]:
import numpy as np 
from week1 import  PiInv, Pi,project_points,plot3d
from scipy.spatial.transform import Rotation


K = np.array([[1000,0,300],[0,1000,200],[0,0,1]])
R_1 = np.eye(3) 
t_1 = 0 

R_2 =  Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix()
t_2 = np.array([0.2,2,1])



3.1 

In [16]:
Q = np.array([1,0.5,4,1])

q1   = K @ np.column_stack((R_1,np.array([0,0,0]))) @ (Q)
q2 = K @ np.column_stack((R_2,t_2)) @ Q


## 3.2

In [17]:
def crossOp(p): 
    p_cross = np.array([[0,-p[2],p[1]],[p[2],0,-p[0]],[-p[1],p[0],0]]) 
    return p_cross

In [18]:
test_1 = np.array([2,4,6])
test_2 = np.array([3,6,8])

np.testing.assert_equal(np.cross(test_1,test_2),crossOp(test_1) @ test_2)

## Exercise 3.3
* Copmute the fundamental matrix of the two cameras 

In [19]:
from sympy import Matrix

E = crossOp(t_2) @ R_2

F = np.linalg.inv(K).T @ E @ np.linalg.inv(K)

Matrix(F)

⎡ 3.2931188116914e-7   8.19396326710898e-7   0.00179162591798073⎤
⎢                                                               ⎥
⎢5.15532551154256e-7   -8.76915984132189e-7  9.31426656231631e-5⎥
⎢                                                               ⎥
⎣-0.00129882755312393  0.00151951700173537    -1.10072682336132 ⎦

## Exercise 3.4
* What is the epipolar line l of q1 in camera two?

q1

In [20]:
l = (F@q1)

print(l)

[ 8.95620504e-03  3.66751496e-04 -5.28495581e+00]


## Exercise 3.5 
* Is q2 located on the epipolar line from Exercise 3.4? Do the computations, but also explain why
this must be so.

In [21]:
q2.T @ l

## ex 3.8



In [22]:
data = np.load('TwoImageDataCar.npy',allow_pickle=True).item()

In [23]:
data.keys()

dict_keys(['im1', 'R1', 't1', 'im2', 'R2', 't2', 'K'])

In [40]:
K = data["K"] 

im1 = data["im1"]
R1 = data["R1"]
t1 = data["t1"] 

im2 = data["im2"]
R2 = data["R2"]
t2 = data["t2"] 

In [46]:
E = crossOp(t1) @ R1
F = np.transpose(np.linalg.inv(K)) @ E @ np.linalg.inv(K) 
Matrix(F)

⎡-1.94029315462207e-8  -1.26535830581291e-6  1.42215454159821e-5 ⎤
⎢                                                                ⎥
⎢-1.74360790570005e-6  -3.16893157425425e-7  -0.00120302873104124⎥
⎢                                                                ⎥
⎣-5.87867477285393e-5   0.0027814667251453    -0.101653094437382 ⎦

In [26]:
import matplotlib.pyplot as plt 

def show2imgs(im1,im2):
    ig,ax = plt.subplots(1,2,figsize=(10,10))
    
    ax[0].imshow(im1)
    ax[1].imshow(im2) 
    x = plt.ginput(1)
    plt.show()
    point = np.array(x).reshape(-1,1)
    return point


def DrawLine(l, shape):
    #Checks where the line intersects the four sides of the image
    # and finds the two intersections that are within the frame
    def in_frame(l_im):
        q = np.cross(l.flatten(), l_im)
        q = q[:2]/q[2]
        if all(q>=0) and all(q+1<=shape[1::-1]):
          return q
        
    lines = [[1, 0, 0], [0, 1, 0], [1, 0, 1-shape[1]], [0, 1, 1-shape[0]]]
    P = [in_frame(l_im) for l_im in lines if in_frame(l_im) is not None]
    if (len(P)==0):
        print("Line is completely outside image")
    plt.plot(*np.array(P).T)

In [33]:
%matplotlib qt 

def ex9(im1,im2,K,R1,R2,t1,t2):
    q1_ = show2imgs(im1,im2)

    
    E = crossOp(t1) @ R1 

    F = np.linalg.inv(K).T @ E @  np.linalg.inv(K) 



    l = F @ PiInv(q1_)
    shape = im2.shape
    
    DrawLine(l,shape)

    return F



In [48]:
F = ex9(im1,im2,K,R1,R2,t1,t2)

In [49]:
Matrix(F)

⎡-5.82322057040713e-8  -2.72441774726627e-6  5.16847341221962e-5 ⎤
⎢                                                                ⎥
⎢-2.96383945823227e-6  -2.80525900143955e-7  -0.00110672213830915⎥
⎢                                                                ⎥
⎣-5.17753981754951e-5   0.0037662029528016    -0.12117903096092  ⎦

# Exercise 3.11

In [176]:
def triangulate(qn,pn):

    """ Input 
        Q: list of n pixel coordinates (q1,q2,...qn)
        Pn: list of n projection matrices 
        output: triangulation of the 3d point
    """

    if len(Q) != len(pn):
        raise ValueError("Expected lists of equal length, len(Q)!=len(pn)")

    
    B_i = lambda P,q: np.array([  [P[2]*q[0] - P[0]],[P[0]*q[1] - P[1]]  ])

    B = np.hstack(([B_i(P_i,q_i) for P_i, q_i in zip(pn,qn)]))

    B = B.reshape(len(pn)*2,4)

    U,S,VT = np.linalg.svd(B)

    Q = VT[-1,:]
    return Q

    



In [91]:
np.empty((2,3))

array([[4.0e-323, 2.0e-323, 1.5e-323],
       [4.9e-324, 4.4e-323, 3.0e-323]])

In [88]:
B

array([[-0.09205746, -0.82851716, -0.55234477],
       [-0.82851716,  0.3714244 , -0.4190504 ],
       [-0.55234477, -0.4190504 ,  0.72063307]])

In [60]:
m1 = np.array([[3,2,3],[2,4,3],[5,1,2]]) 
m2 = np.array([[3,2,3],[6,4,3],[6,2,1]]) 
m3 =  np.array([[3,2,8],[4,3,6],[8,10,5]]) 


lst = [m1,m2]
lst[0][0]

qn = np.array([[1,2],[4,6]])

In [67]:
B = np.array([  [lst[0][2]*qn[0][0]-lst[0][0]],[lst[0][2]*qn[0][1] - lst[0][1]]  ])

In [68]:
B 

array([[[ 2, -1, -1]],

       [[ 8, -2,  1]]])

In [70]:
lst[0][2]

array([5, 1, 2])

In [65]:
B

array([[4, 0, 1]])